In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
df = pd.read_csv('/kaggle/input/houkong-moai/customer_churn.csv')
df = df.drop_duplicates()

In [ ]:
def create_features(df):
    # 添加相关特征
    df['avg_monthly_usage'] = df['total_usage'] / df['subscription_length']
    df['bill_usage_ratio'] = df['monthly_bill'] / df['total_usage']
    df['complaint_frequency'] = df['service_complaints'] / df['subscription_length']
    return df

df = create_features(df)

In [ ]:
X = df.drop(['customer_id', 'churn'], axis=1)
y = df['churn']

In [ ]:
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['number']).columns

preprocessor = ColumnTransformer([
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_cols),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())]), numerical_cols)
])

In [ ]:
pipeline = Pipeline([
    ('prep', preprocessor),
    ('clf', RandomForestClassifier(
        class_weight='balanced',
        random_state=42,
        n_jobs=-1))
])

In [ ]:
params = {
    'clf__n_estimators': [200, 300],
    'clf__max_depth': [5, 30],
    'clf__min_samples_leaf': [2, 3]
}

In [ ]:
searcher = GridSearchCV(pipeline, params, cv=5, scoring='accuracy')
searcher.fit(X, y)

In [ ]:
best_model = searcher.best_estimator_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = best_model.predict(X_test)

print(f"最佳参数: {searcher.best_params_}")
print(f"测试集准确率: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

In [ ]:
test_predictions = best_model.predict(X)
submission = pd.DataFrame({
    'ID': df['customer_id'],
    'Churn': test_predictions
})
submission.to_csv('/kaggle/working/submission.csv', index=False)